In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用TGI将Gemma部署到GKE

<table align="left"><tbody><tr>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_gemma_deployment_on_gke.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> 在Colab Enterprise中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_deployment_on_gke.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      在GitHub上查看
    </a>
  </td>
</tr></tbody></table>

## 概述

本笔记本演示了使用文本生成推理[TGI](https://github.com/)从Google DeepMind下载和部署Gemma开放模型的方法，这是一种提高服务吞吐量的有效服务选项。在本笔记本中，我们将在GPU上部署和提供TGI。在本指南中，我们专门使用L4 GPU，但是这个指南也适用于A100(40GB)、A100(80GB)、H100(80GB) GPU。


### 目标

在GPU上部署和运行用于提供Gemma的TGI的推理。

### GPU

GPU能够加速在节点上运行的特定工作负载，如机器学习和数据处理。GKE提供各种机器类型选项，用于节点配置，包括带有NVIDIA H100、L4和A100 GPU的机器类型。

在GKE中使用GPU之前，我们建议您完成以下学习路径：

了解[当前GPU版本的可用性](https://cloud.google.com/compute/docs/gpus)

了解[GKE中的GPU](https://cloud.google.com/kubernetes-engine/docs/concepts/gpus)


### TGI

TGI是一个高度优化的开源LLM服务框架，可以提高GPU上的服务吞吐量。TGI具有以下特性：

优化的Transformer实现与PagedAttention
连续的批处理以提高整体服务吞吐量
张量并行和在多个GPU上的分布式服务

要了解更多信息，请参考[TGI文档](https://github.com/huggingface/text-generation-inference/blob/main/README.md)

## 运行笔记本

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. Set Hugging Face access token in `HF_TOKEN` field. If you don't already have a "read" access token, follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create an access token with "read" permission. You can find your existing access tokens in the Hugging Face [Access Token](https://huggingface.co/settings/tokens) page.

# @markdown 3. **[Optional]** Set `CLUSTER_NAME` if you want to use your own GKE cluster. If not set, this example will create a standard cluster with 2 NVIDIA L4 GPU accelerators.

import os
from datetime import datetime

# The HuggingFace token used to download models.
HF_TOKEN = ""  # @param {type:"string"}
assert HF_TOKEN, "Please set Hugging Face access token in `HF_TOKEN`."

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Set up gcloud.
! gcloud config set project "$PROJECT_ID"
! gcloud services enable container.googleapis.com

# Add kubectl to the set of available tools.
! mkdir -p /tools/google-cloud-sdk/.install
! gcloud components install kubectl --quiet

# The cluster name to create
CLUSTER_NAME = ""  # @param {type:"string"}

# Use existing GKE cluster or create a new cluster.
if CLUSTER_NAME:
    ! gcloud container clusters get-credentials {CLUSTER_NAME} --location {REGION}
else:
    now = datetime.now().strftime("%Y%m%d%H%M%S")
    CLUSTER_NAME=f"gke-gemma-cluster-{now}"
    ! gcloud container clusters create {CLUSTER_NAME} \
        --project={PROJECT_ID} \
        --region={REGION} \
        --workload-pool={PROJECT_ID}.svc.id.goog \
        --release-channel=rapid \
        --num-nodes=4
    ! gcloud container node-pools create gpupool \
        --accelerator=type=nvidia-l4,count=2,gpu-driver-version=latest \
        --project={PROJECT_ID} \
        --location={REGION} \
        --node-locations={REGION}-a \
        --cluster={CLUSTER_NAME} \
        --machine-type=g2-standard-24 \
        --num-nodes=1

# Create Kubernetes secret for Hugging Face credentials
! kubectl create secret generic hf-secret \
    --from-literal=hf_api_token={HF_TOKEN} \
    --dry-run=client -o yaml > hf-secret.yaml

! kubectl apply -f hf-secret.yaml

In [ ]:
# @title Deploy TGI

# @markdown This section deploys Gemma on TGI.

# @markdown Select one of the following model version and size options:

# The size of the model to launch
MODEL_VERSION = "1.1"  # @param ["1.0", "1.1"]
if MODEL_VERSION == "1.1":
    version_string = "-1.1"
else:
    version_string = ""
# The size of the model to launch
MODEL_SIZE = "2b"  # @param ["2b", "7b"]

# @markdown After the container is up, there will be another ~5 minutes to download the needed artifacts, the time depends on what runtime you are using to run your colab environment.

# The number of GPUs to run: 1 for 2b, 2 for 7b
GPU_COUNT = 1
if MODEL_SIZE == "7b":
    GPU_COUNT = 2

# Ephemeral storage
EPHEMERAL_STORAGE_SIZE = "20Gi"
if MODEL_SIZE == "7b":
    EPHEMERAL_STORAGE_SIZE = "40Gi"

# Memory size
MEMORY_SIZE = "7Gi"
if MODEL_SIZE == "7b":
    MEMORY_SIZE = "25Gi"

GPU_SHARD = 1
if MODEL_SIZE == "7b":
    GPU_SHARD = 2

CPU_LIMITS = 2
if MODEL_SIZE == "7b":
    CPU_LIMITS = 10

K8S_YAML = f"""
apiVersion: apps/v1
kind: Deployment
metadata:
  name: tgi-gemma-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: gemma-server
  template:
    metadata:
      labels:
        app: gemma-server
        ai.gke.io/model: gemma{version_string}-{MODEL_SIZE}
        ai.gke.io/inference-server: text-generation-inference
        examples.ai.gke.io/source: user-guide
    spec:
      containers:
      - name: inference-server
        image: us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-hf-tgi-serve:20240220_0936_RC01
        resources:
          requests:
            cpu: "2"
            memory: {MEMORY_SIZE}
            ephemeral-storage: {EPHEMERAL_STORAGE_SIZE}
            nvidia.com/gpu: {GPU_COUNT}
          limits:
            cpu: {CPU_LIMITS}
            memory: {MEMORY_SIZE}
            ephemeral-storage: {EPHEMERAL_STORAGE_SIZE}
            nvidia.com/gpu: {GPU_COUNT}
        args:
        - --model-id=$(MODEL_ID)
        - --num-shard={GPU_SHARD}
        env:
        - name: MODEL_ID
          value: google/gemma{version_string}-{MODEL_SIZE}-it
        - name: PORT
          value: "8000"
        - name: HUGGING_FACE_HUB_TOKEN
          valueFrom:
            secretKeyRef:
              name: hf-secret
              key: hf_api_token
        volumeMounts:
        - mountPath: /dev/shm
          name: dshm
      volumes:
      - name: dshm
        emptyDir:
          medium: Memory
      nodeSelector:
        cloud.google.com/gke-accelerator: nvidia-l4
---
apiVersion: v1
kind: Service
metadata:
  name: llm-service
spec:
  selector:
    app: gemma-server
  type: ClusterIP
  ports:
  - protocol: TCP
    port: 8000
    targetPort: 8000
"""

with open("tgi.yaml", "w") as f:
    f.write(K8S_YAML)

! kubectl apply -f tgi.yaml

# Wait for container to be created.
import time

print("Waiting for container to be created...\n")
while True:
    shell_output = ! kubectl get pod
    container_status = "\n".join(shell_output)
    if "1/1" in container_status:
        break
    time.sleep(5)

print(container_status)

# Wait for downloading artifacts.
print("\nDownloading artifacts...")
while True:
    shell_output = ! kubectl logs -l app=gemma-server
    logs = "\n".join(shell_output)
    if "Connected" in logs:
        break
    time.sleep(5)

print("Server is up and running.")

In [ ]:
# @title Prediction

# @markdown Once the server is up and running, you may send prompts to local server for prediction.

import json

prompt = "What are the top 5 most popular programming languages? Please be brief."  # @param {type: "string"}
temperature = 0.40  # @param {type: "number"}
top_p = 0.1  # @param {type: "number"}
max_tokens = 250  # @param {type: "number"}

request = {
    "inputs": prompt,
    "temperature": temperature,
    "top_p": top_p,
    "max_tokens": max_tokens,
}

command = f"""kubectl exec -t $( kubectl get pod -l app=gemma-server -o jsonpath="{{.items[0].metadata.name}}" ) -c inference-server -- curl -X POST http://localhost:8000/generate \
   -H "Content-Type: application/json" \
   -d '{json.dumps(request)}' \
   2> /dev/null"""

output = !{command}
print("Output:")
print(json.loads(output[0])["generated_text"])

In [ ]:
# @title Clean up resources

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continouous charges that may incur.

! kubectl delete deployments tgi-gemma-deployment
! kubectl delete services llm-service
! kubectl delete secrets hf-secret

DELETE_CLUSTER = False # @param {type: "boolean"}

if DELETE_CLUSTER:
  ! gcloud container clusters delete {CLUSTER_NAME} \
    --region={REGION} \
    --quiet